## Installing Sentence Transsformer and other models/frameworks

In [1]:
!pip install sentence_transformers
!pip install scipy
!pip install gensim
!pip install nltk
!pip install spacy

# Kindly add all your installations and versions if any in this cell.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.0 MB/s 
     |████████████████████████████████| 5.5 MB 41.7 MB/s 
     |████████████████████████████████| 1.3 MB 45.0 MB/s 
     |████████████████████████████████| 182 kB 51.2 MB/s 
     |████████████████████████████████| 7.6 MB 15.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=59c67fb5d7f3dbf4551d7c5f276059bda7adee7e5974fbc1abd5b9f1dc4eeebc
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pu

## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [2]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
import requests
import tarfile
import csv
import string

from sklearn.svm import SVR

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import gensim
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [3]:
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
    """
    Take the input path and return the dataframe
    """
    url = 'http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz'
    target_path = 'Stsbenchmark.tar.gz'
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())
    file = tarfile.open(target_path)
    file.extractall('D:/STS')
    file.close()
    path = "D:/STS/stsbenchmark/sts-" + dataset_type + ".csv"
    df = pd.read_csv(path, sep='\t', quoting = csv.QUOTE_NONE,
                    encoding='utf-8', error_bad_lines=False)
    df.columns = columns
    return df

df_train = read_sts_csv("train") # create the train, dev and test dataframes
df_test = read_sts_csv("test")
df_dev = read_sts_csv("dev")

# Printing the final shapes of the dataframes
print(df_train.shape, df_test.shape, df_dev.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


b'Skipping line 5553: expected 7 fields, saw 9\nSkipping line 5554: expected 7 fields, saw 9\nSkipping line 5555: expected 7 fields, saw 9\nSkipping line 5556: expected 7 fields, saw 9\nSkipping line 5557: expected 7 fields, saw 9\nSkipping line 5558: expected 7 fields, saw 9\nSkipping line 5559: expected 7 fields, saw 9\nSkipping line 5560: expected 7 fields, saw 9\nSkipping line 5561: expected 7 fields, saw 9\nSkipping line 5562: expected 7 fields, saw 9\nSkipping line 5563: expected 7 fields, saw 9\nSkipping line 5564: expected 7 fields, saw 9\nSkipping line 5565: expected 7 fields, saw 9\nSkipping line 5566: expected 7 fields, saw 9\nSkipping line 5567: expected 7 fields, saw 9\nSkipping line 5568: expected 7 fields, saw 9\nSkipping line 5569: expected 7 fields, saw 9\nSkipping line 5570: expected 7 fields, saw 9

(5551, 7) (1094, 7) (1477, 7)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


b'Skipping line 1479: expected 7 fields, saw 9\nSkipping line 1480: expected 7 fields, saw 9\nSkipping line 1481: expected 7 fields, saw 9\nSkipping line 1482: expected 7 fields, saw 9\nSkipping line 1483: expected 7 fields, saw 9\nSkipping line 1484: expected 7 fields, saw 9\nSkipping line 1485: expected 7 fields, saw 9\nSkipping line 1486: expected 7 fields, saw 9\nSkipping line 1487: expected 7 fields, saw 9\nSkipping line 1488: expected 7 fields, saw 9\nSkipping line 1489: expected 7 fields, saw 9\nSkipping line 1490: expected 7 fields, saw 9\nSkipping line 1491: expected 7 fields, saw 9\nSkipping line 1492: expected 7 fields, saw 9\nSkipping line 1493: expected 7 fields, saw 9\nSkipping line 1494: expected 7 fields, saw 9\nSkipping line 1495: expected 7 fields, saw 9\nSkipping line 1496: expected 7 fields, saw 9

## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [4]:
# NON_CONEXTUAL_MODEL_TYPE = <NON_CONEXTUAL_MODEL_TYPE>
CONEXTUAL_MODEL_TYPE = 'all-MiniLM-L6-v2'
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = 'bert-base-uncased' # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
# INPUT_PATH = <INPUT_FOLDER_PATH>
BATCH_SIZE = 16
OUT_DIM_DENSE = 256
NUM_EPOCHS = 2 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



In [5]:
def clean_sentences_and_tokenize(sentences: list):
    # Cleaning the data
    sentences_cleaned = []
    for sentence in sentences:
        s = sentence
        # Lowercasing the sentences
        s = s.lower()
        # Removing the punctuations
        punctuations = list(string.punctuation)
        for punc in punctuations:
            if punc in s:
                s = s.replace(punc, ' ')
        s = s.strip()
        # Removing extra space
        s = " ".join(s.split())
        sentences_cleaned.append(s)
    
    data = []
    for sentence in sentences_cleaned:
        # Removing stop words
        stop_words = set(stopwords.words('english'))
        temp = []
        # tokenize the sentence into words
        for j in word_tokenize(sentence):
            if j not in stop_words:
                temp.append(j.strip())
        data.append(temp)
    return data

def get_X_features(feature_1_train, feature_2_train):
    features = []
    for i in range(len(feature_1_train)):
        temp = feature_1_train[i]
        temp.extend(feature_2_train[i])
        features.append(temp)
    return features

def pad_sentences(X_set, max_len):
    padded_sentences = []
    for sentence in X_set:
        x = sentence
        zeros = [0] * (max_len-len(x))
        x.extend(zeros)
        padded_sentences.append(x)
    return padded_sentences

def get_max_min(X_train, X_test, X_dev):
    max_len = 0
    for d in X_train:
        max_len = max(max_len, len(d))
    for d in X_test:
        max_len = max(max_len, len(d))
    for d in X_dev:
        max_len = max(max_len, len(d))
    print("Max_len", max_len)
    
    min_len = 1000000
    for d in X_train:
        min_len = min(min_len, len(d))
    for d in X_test:
        min_len = min(min_len, len(d))
    for d in X_dev:
        min_len = min(min_len, len(d))
    print("Min_len", min_len)
    return max_len, min_len

In [ ]:
def get_feature_model1(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """

# non_cont_model1 = 

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model1(data_frame)
# X_<dataset_type>, Y_<dataset_type> = 
# Initiate a regression model and train it.
# Print spearmanr correlation on the predicted output of the dev and test sets.

In [6]:
def get_feature_model1(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    # embedding_a, embedding_b = process(data_frame)
    # return embedding_a, embedding_b
    sentences_a = data_frame['sent_a'].to_list()
    sentences_b = data_frame['sent_b'].to_list()
    data_a = clean_sentences_and_tokenize(sentences_a)
    data_b = clean_sentences_and_tokenize(sentences_b)
    # For new version of Word2Vec
    # model_a = gensim.models.Word2Vec(data_a, min_count = 1, vector_size = 100, window = 5, sg=0)
    # model_b = gensim.models.Word2Vec(data_b, min_count = 1, vector_size = 100, window = 5, sg=0)

    # For old version of Word2Vec (Required when running on Colab)
    model_a = gensim.models.Word2Vec(data_a, min_count = 1, size = 100, window = 5, sg=0)
    model_b = gensim.models.Word2Vec(data_b, min_count = 1, size = 100, window = 5, sg=0)
    
    sentence_embeddings_a = []
    for sentence in data_a:
        sen_vec = []
        for word in sentence:
            sen_vec.extend(model_a.wv[word])
        sentence_embeddings_a.append(sen_vec)
        
    sentence_embeddings_b = []
    for sentence in data_b:
        sen_vec = []
        for word in sentence:
            sen_vec.extend(model_b.wv[word])
        sentence_embeddings_b.append(sen_vec)
    
    return sentence_embeddings_a, sentence_embeddings_b

    
feature_1_train, feature_2_train = get_feature_model1(df_train)
feature_1_test, feature_2_test = get_feature_model1(df_test)
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)

X_train = get_X_features(feature_1_train, feature_2_train)
X_test = get_X_features(feature_1_test, feature_2_test)
X_dev = get_X_features(feature_1_dev, feature_2_dev)

max_len, min_len = get_max_min(X_train, X_test, X_dev)

X_train, Y_train = pad_sentences(X_train, max_len), df_train['score']
X_test, Y_test = pad_sentences(X_test, max_len), df_test['score']
X_dev, Y_dev = pad_sentences(X_dev, max_len), df_dev['score']

get_max_min(X_train, X_test, X_dev)

# Initiate a regression model and train it
svr_model = SVR()
svr_model.fit(X_train, Y_train)

Y_test_pred = np.array(svr_model.predict(X_test))
Y_dev_pred = np.array(svr_model.predict(X_dev))

# Print spearmanr correlation on the predicted output of the dev and test sets.
print(f"Test spearmanr Correlation: {stats.spearmanr(Y_test_pred, Y_test)}")
print(f"Dev spearmanr Correlation: {stats.spearmanr(Y_dev_pred, Y_dev)}")

Max_len 6000
Min_len 200
Max_len 6000
Min_len 6000
Test spearmanr Correlation: SpearmanrResult(correlation=0.24379149426414087, pvalue=2.8847701506016075e-16)
Dev spearmanr Correlation: SpearmanrResult(correlation=0.11459175058693614, pvalue=1.0111114242612242e-05)


## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
def get_feature_model2(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using model2,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """

# non_cont_model2 = 

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
# X_<dataset_type>, Y_<dataset_type> = 
# Initiate a regression model and train it.
# Print spearman correlation on the predicted output of the dev and test sets.

In [7]:
def get_feature_model2(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using model2,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    non_cont_model2 = SentenceTransformer(CONEXTUAL_MODEL_TYPE)
    sentences1 = data_frame['sent_a'].to_list()
    sentences2 = data_frame['sent_b'].to_list()
    embeddings1 = non_cont_model2.encode(sentences1)
    embeddings2 = non_cont_model2.encode(sentences2)
    print(embeddings1.shape, embeddings2.shape)
    return embeddings1, embeddings2


feature_1_train, feature_2_train = get_feature_model2(df_train)
feature_1_test, feature_2_test = get_feature_model2(df_test)
feature_1_dev, feature_2_dev = get_feature_model2(df_dev)

X_train, Y_train = np.concatenate((feature_1_train, feature_2_train), axis=1), df_train['score']
X_test, Y_test = np.concatenate((feature_1_test, feature_2_test), axis=1), df_test['score']
X_dev, Y_dev = np.concatenate((feature_1_dev, feature_2_dev), axis=1), df_dev['score']

# Initiate a regression model and train it.
svr_model = SVR(C=1.0, epsilon=0.2)
svr_model.fit(X_train, Y_train)

Y_test_pred = np.array(svr_model.predict(X_test))
Y_dev_pred = np.array(svr_model.predict(X_dev))

# Print spearman correlation on the predicted output of the dev and test sets.
print(f"Test spearmanr Correlation: {stats.spearmanr(Y_test_pred, Y_test)}")
print(f"Dev spearmanr Correlation: {stats.spearmanr(Y_dev_pred, Y_dev)}")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

(5551, 384) (5551, 384)
(1094, 384) (1094, 384)
(1477, 384) (1477, 384)
Test spearmanr Correlation: SpearmanrResult(correlation=0.5694996874487458, pvalue=4.58734535971537e-95)
Dev spearmanr Correlation: SpearmanrResult(correlation=0.4981336322714802, pvalue=1.8684863190089402e-93)


## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [ ]:
def form_data(data_frame):
  """
  Input a data frame and return the dataloder.
  """

def get_model_predicts(data_type, trained_model):
  """
  Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
  """

# dataloader_<dataset_type> = form_data(data_frame)
# base_model = 
# layer_ppoling = 
# layer_dense = 
# model3 = 
# loss =

# Fit the model3.
# Print spearman correlation on the predicted output of the dev and test sets.

In [23]:
def form_data(data_frame):
    """
    Input a data frame and return the dataloder.
    """
    texts1 = data_frame['sent_a'].to_list()
    texts2 = data_frame['sent_b'].to_list()
    labels = data_frame['score'].to_list()
    labels = [i / 5 for i in labels]
    examples = []
    for i in range(len(texts1)):
        in_exp = InputExample(texts=[texts1[i], texts2[i]], label=labels[i])
        examples.append(in_exp)
    dataloader = DataLoader(examples, shuffle=True, batch_size=BATCH_SIZE)
    return dataloader

def get_model_predicts(data_type, trained_model):
    """
    Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
    """
    sentences1 = data_type['sent_a'].to_list()
    sentences2 = data_type['sent_b'].to_list()

    #Compute embedding for both lists
    embeddings1 = trained_model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = trained_model.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    
    scores = []
    for i in range(len(cosine_scores)):
        scores.append(cosine_scores[i][i])
        
    return scores

dataloader_train = form_data(df_train)
dataloader_test = form_data(df_test)
dataloader_dev = form_data(df_dev)

base_model = models.Transformer(HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL, max_seq_length=256)
layer_pooling = models.Pooling(base_model.get_word_embedding_dimension())
layer_dense = models.Dense(in_features=layer_pooling.get_sentence_embedding_dimension(), out_features=OUT_DIM_DENSE, activation_function=nn.Tanh())
model3 = SentenceTransformer(modules=[base_model, layer_pooling, layer_dense])
loss = losses.CosineSimilarityLoss(model3)

# Fit the model3.
model3.fit(train_objectives=[(dataloader_train, loss)], epochs=NUM_EPOCHS, warmup_steps=100)

# Print spearman correlation on the predicted output of the dev and test sets.
test_scores = get_model_predicts(df_test, model3)
dev_scores = get_model_predicts(df_dev, model3)

# Run these two lines of code when running on GPU otherwise comment them out.
# These lines help in transferring the outputs from the CUDA environment to the CPU environment
test_scores = [test_scores[i].cpu().detach().numpy().item() for i in range(len(test_scores))]
dev_scores = [dev_scores[i].cpu().detach().numpy().item() for i in range(len(dev_scores))]

print(f"Test spearmanr Correlation: {stats.spearmanr(test_scores, df_test['score'])}")
print(f"Dev spearmanr Correlation: {stats.spearmanr(dev_scores, df_dev['score'])}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/347 [00:00<?, ?it/s]

Iteration:   0%|          | 0/347 [00:00<?, ?it/s]

Test spearmanr Correlation: SpearmanrResult(correlation=0.8510036907993377, pvalue=1.0253049835325059e-307)
Dev spearmanr Correlation: SpearmanrResult(correlation=0.8695100621765706, pvalue=0.0)
